In [343]:
import COVID19Py
import pandas


In [359]:
# Import from S3:
jhu_df = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/jhu_df.csv.gz', index_col=0)
jhu_df_time = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/jhu_df_time.csv.gz', index_col=0)
csbs_df = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/csbs_df.csv.gz', index_col=0)
per_day_stats_by_country = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/country_df_per_day.csv.gz', index_col=0)
per_day_stats_by_state = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/provence_df_per_day.csv.gz', index_col=0)

jhu_df_time['Date'] = pd.to_datetime(jhu_df_time['Date'])
jhu_df['Date'] = pd.to_datetime(jhu_df['Date'])
csbs_df['Date'] = pd.to_datetime(csbs_df['Date'])
jhu_df['state'] = ""


date_mapper = pd.DataFrame(
    jhu_df_time['Date'].unique(), columns=['Date'])
date_mapper['Date_text'] = date_mapper['Date'].dt.strftime('%m/%d/%y')
min_date = date_mapper.index[0]
max_date = date_mapper.index[-1]


merge = pd.concat([jhu_df, csbs_df])
merge = merge.fillna('')
problem_countries = merge[merge['country_code'] == '']['country'].tolist()
merge.loc[merge['country'] == 'Namibia', 'country_code'] = ''
centroid_mapper = pd.read_csv('country_centroids_az8.csv')
problem_states = merge[~merge['country_code'].isin(centroid_mapper['iso_a2'])]
new_merge = merge.merge(
    centroid_mapper, left_on='country_code', right_on='iso_a2')
new_merge['case_rate'] = new_merge['confirmed']/new_merge['pop_est'] * 100
new_merge['death_rate'] = new_merge['deaths']/new_merge['pop_est'] * 100
new_merge['confirmed_no_death'] = new_merge['confirmed'] - new_merge['deaths']

# If something is has the same name for continent and subregion, lets just add the word _subregion
new_merge.loc[new_merge['continent'] == new_merge['subregion'],
              'subregion'] = new_merge['subregion'] + ' Subregion'



,id,lat,lon,Date,province,country_code,country,confirmed,deaths,recovered,...,name_len,long_len,abbrev_len,tiny,homepart,Longitude,Latitude,case_rate,death_rate,confirmed_no_death
0,0,33.000000,65.000000,2020-03-24 06:01:08.402656+00:00,,AF,Afghanistan,40,1,0,...,11,11,4,-99,1,66.004734,33.835231,0.000141,0.000004,39
1,1,41.153300,20.168300,2020-03-24 06:01:08.412527+00:00,,AL,Albania,104,4,0,...,7,7,4,-99,1,20.049834,41.142450,0.002858,0.000110,100
2,2,28.033900,1.659600,2020-03-24 06:01:08.417963+00:00,,DZ,Algeria,230,17,0,...,7,7,4,-99,1,2.617323,28.158938,0.000673,0.000050,213
3,3,42.506300,1.521800,2020-03-24 06:01:08.423338+00:00,,AD,Andorra,133,1,0,...,7,7,4,5,1,1.560544,42.542291,0.158545,0.001192,132
4,4,-11.202700,17.873900,2020-03-24 06:01:08.428696+00:00,,AO,Angola,3,0,0,...,6,6,4,-99,1,17.537368,-12.293361,0.000023,0.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504,233,12.116500,-61.679000,2020-03-24 06:01:09.466806+00:00,,GD,Grenada,1,0,0,...,7,7,5,-99,1,-61.682202,12.117250,0.001102,0.000000,1
1505,234,-18.665695,35.529562,2020-03-24 06:01:09.471494+00:00,,MZ,Mozambique,1,0,0,...,10,10,4,-99,1,35.533675,-17.273816,0.000005,0.000000,1
1506,235,34.802075,38.996815,2020-03-24 06:01:09.475908+00:00,,SY,Syria,1,0,0,...,5,5,5,-99,1,38.507882,35.025474,0.000005,0.000000,1
1507,236,-8.874217,125.727539,2020-03-24 06:01:09.480471+00:00,,TL,Timor-Leste,1,0,0,...,11,11,4,-99,1,125.844390,-8.828892,0.000088,0.000000,1


In [351]:
 COVID19Py.COVID19(data_source="jhu").getAll(timelines)

{'latest': {'confirmed': 418678, 'deaths': 18625, 'recovered': 0},
 'locations': []}